# Biotrickling Filter Mass Balance
## Gas Phase
Odorous compounds initially enter the biotrickling filter in the gas phase. As they travel through the column, these components will be in contact with the trickling liquid phase, where mass transfer occurs due to the concentration gradient between the two phases.

The following assumptions are made:


<li> The trickling liquid uniformly covers the biofilm. Its contact area can thus be expressed using the bed void fraction. </li>
<li> The void fraction is assumed to be constant throughout the column. </li>
<li> Plug flow is assumed, so mass transport is predominantly convective. </li>
<li> The superficial velocity is constant throughout the column. </li>

Using these assumptions, a microscopic model is developed over a differential volume of the column. The vertical axis is represented by the variable 𝑧.

$$ \epsilon \frac{\delta C_g}{\delta t} = -V_0 \frac{\delta C_g}{\delta z} - (1 - \epsilon) R_g$$

<!-- Shinabe et al. developed a model for the reaction rate.

$$ R_g = -K_La(\frac{C_g}{m} - C_L) $$

Where $K_La$ is the volumetric mass transfer coefficient, $C_g$ is the gas phase concentration, $C_L$ is the liquid phase concentration, and $m$ is solubility constant.

The volumetric mass transfer coefficient was determined by using a cell suspension under the $H_2S$ diffusion-limiting condition.

$$K_La = -\frac{V_g m}{V_L  t} ln(\frac{C_g}{C_{g0}}) = -\frac{\epsilon m}{(1 - \epsilon)  t} ln(\frac{C_g}{C_{g0}})$$

## Liquid Phase -->

<!-- To begin, we will neglect the liquid phase and assume that $R_g$ is equal to the total substrate consumption rate in the bacteria.

$$ R_g = -\frac{dS}{dt} = -q_sX = -\frac{dX/dt}{Y_{X/S}} - \frac{dP/dt}{Y_{P/S}} - mX$$ -->
